In [9]:
import numpy as np
import progressbar 
import random 
import os
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

import keras
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt
%matplotlib inline

from helpers import HDF5DatasetWriter
from helpers import Utils

In [18]:
output_path = "datasets/UCF-101-hdf5/fixed-sequance-rgbmean.hdf5"

In [15]:
import pickle
data = pickle.load(open('../input/datasets/UCF-101-hdf5/full_sequence.pkl', 'rb'))
rgbmean = pickle.load(open('../input/datasets/UCF-101-hdf5/rgbmean.pkl', 'rb'))

In [16]:
rgbmean

{'B': 83.127605197057377, 'G': 94.267999972782135, 'R': 100.86576035056612}

In [12]:
x, y = np.array([t["imgs"] for t in data]), [t["label"] for t in data]
x = pad_sequences(x, maxlen=20, truncating="post")

lb = LabelBinarizer()
y = lb.fit_transform(y)

In [19]:
def export_rgbmean_data(x, y, output_path, buffer_size):
    dataset = HDF5DatasetWriter(x.shape, y.shape, output_path, bufSize = buffer_size)
    
    widgets = ["Exporting Data: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
    pbar = progressbar.ProgressBar(maxval=x.shape[0], widgets=widgets).start()
    
    sequences = []
    for i, sequence in enumerate(x):
        images = []
        for i, image in enumerate(sequence):
            (B, G, R) = cv2.split(image.astype("float32"))
            R -= rgbmean["R"]
            G -= rgbmean["G"]
            B -= rgbmean["B"]
            image = cv2.merge([B, G, R])
            image = img_to_array(image)
            image = imagenet_utils.preprocess_input(image)
            images.append(image)
        sequences.append(images)
        pbar.update(i)
    dataset.add(sequences, y)
    dataset.close()
    pbar.finish()

In [21]:
export_rgbmean_data(x, y, output_path, 1000)

Exporting Data: 100% |##########################################| Time: 0:00:00
